In [1]:
# Imports for scraping RT
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from collections import defaultdict


import re

# from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import requests


# Imports for data formatting
import uuid
import hashlib
import json
from datetime import datetime, date


In [2]:
from bs4 import BeautifulSoup
import time
import requests
def get_soup(url):
    try:
        time.sleep(1)
        response = requests.get(url)
        if response.url[-5:] == url[-5:]:
            soup = BeautifulSoup(response.text, 'html.parser')
            return soup
    except Exception as e:
        print("url  " + url + " raised exception " + str(e))
        return ""


soup = get_soup("https://ew.com/movie-reviews/2019/10/02/steven-soderbergh-laundromat-movie-review/")
print(soup)
# requests.get("https://screenrant.com/when-you-finish-saving-the-world-2023-movie-reviews/")


<!DOCTYPE html>

<html class="comp no-js taxlevel-2 structuredContentTemplate article-html html mntl-html" data-ab="99,99,99,99,99,99,99" data-ew-resource-version="1.12.0" data-mantle-resource-version="3.14.318" data-resource-version="1.12.0" data-tracking-container="true" id="structuredContentTemplate_1-0" lang="en">
<!--
<globe-environment environment="k8s-prod" application="ew" dataCenter="us-west-1"/>
-->
<head class="loc head">
<script type="text/javascript">var Mntl = window.Mntl || {};</script>
<link href="//js-sec.indexww.com" rel="preconnect"/>
<link href="//c.amazon-adsystem.com" rel="preconnect"/>
<link href="//securepubads.g.doubleclick.net" rel="preconnect"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="max-image-preview:large, NOODP, NOYDIR" name="robots">
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<link href="https://ew.com/movie-reviews/2019/10/02/steven-soderbergh-laundromat-movie-review/" re

In [3]:
def scrape_soup(soup):
    """
    Scrapes and process text from a webpage, focusing on <p> tags.

    Parameters:
    url (str): The URL of the webpage to scrape.

    Returns:
    str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
    """
    
    # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
    def get_class(p):
        return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"
    
    p_dict = defaultdict(list)
    try:
        for element in soup.find_all():
            if element.name == "header" or element.name == "style" or element.name == "blockquote":
                element.decompose()
            elif element.attrs and 'class' in element.attrs:
                if any("fl-node-aou15irpt4cb" in class_name.lower() 
                        or "podcast__wrapper" in class_name.lower()
                    for class_name in element['class']):
                        # print("slice-author-bio found")
                        for sibling in element.find_all_next():
                            sibling.decompose()
                        element.decompose()
                        break
                elif any("caption" in class_name.lower()
                    for class_name in element['class']):
                    element.decompose()
            elif element.text and (element.text.startswith("Cast:")):
                element.find_next_sibling('p').decompose()
                break

            # elif element.name == "hr" or (element.text and element.text.startswith("Director:")):
            #     # Perform actions on the element with the specific id
            #     for sibling in element.find_all_next():
            #         sibling.decompose()
            #     element.decompose()
                # break

            # elif element.name == "footer" or element.name == "h4":
            #     for sibling in element.find_all_next():
            #         sibling.decompose()
            #     element.decompose()

                        
        # finds all <p> tags
        paragraphs = soup.find_all('p')
        for p in paragraphs:
            if len(p.text.split(' ')) > 5 and not p.find(['ol', 'ul', 'dl' ]) and not p.text.startswith("Director"):
                tags= p.find_all(['i', 'em', 'strong']) 
                if len(tags) == 1 and tags[0].text == p.text:
                    continue
                if p.find('a'):
                    # Check if the anchor text is the same as the paragraph text
                    if p.find('a').text == p.text:
                        continue 
                p_dict[get_class(p)].append(p.text)
        # print(p_dict)
        # Find the class with the longest list of paragraphs
        longest_key = max(p_dict, key=lambda k: len(p_dict[k]))
        # Combine and clean text
        full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()

    except Exception as e:
        print("Exception "  + str(e) + " occured")
        return ""

    return full_text


In [4]:
print(scrape_soup(soup))


Praise or blame The Big Short for what its success has wrought: the unchecked rise of the winky, starry movie-splainer, in which a typically dry and technical premise — subprime mortgages, international tax law — is recast in kicky colloquialisms and doused liberally with fourth-wall breaks and first-rank celebrities. That bag of tricks is one Short‘s director Adam McKay employed again, a little more queasily, in last year’s loose Dick Cheney biopic Vice, and Steven Soderbergh now borrows with steeply diminishing returns in The Laundromat, a hectic bundle of big names and good intentions whose execution, alas, too often leans toward the skimpy, scattered, and overly glib. Meryl Streep, Sharon Stone, Antonio Banderas, Jeffrey Wright, David Schwimmer, and Gary Oldman are just a few of the actors who appear in the service of exploring the so-called Panama Papers — whose leak to the press in 2015 revealed decades of financial malfeasance by both public and private figures — via various glo

In [5]:
# print(soup)
# 


In [6]:

# # Read the JSONL file and store JSON objects in a list
# with open('/shared/3/projects/hiatus/rotten_tomatoes/rtcorpus.jsonl', 'r') as file:
#     json_list = [json.loads(line) for line in file]

# # Create a set to store unique "fullText" values
# unique_fullText_values = set()

# # Create a new list to store filtered JSON objects
# filtered_json_list = []

# # Iterate through the JSON objects
# for json_obj in json_list:
#     # Extract the "fullText" value
#     fullText_value = json_obj.get("fullText", "")

#     # Check if the "fullText" value is unique
#     if fullText_value not in unique_fullText_values:
#         unique_fullText_values.add(fullText_value)
#         filtered_json_list.append(json_obj)

# # Write the filtered JSON objects back to the JSONL file
# with open('/shared/3/projects/hiatus/rotten_tomatoes/newcorpus.jsonl', 'w') as file:
#     for json_obj in filtered_json_list:
#         file.write(json.dumps(json_obj) + '\n')


In [7]:
# def scrape_soup(soup):
#     """
#     Scrapes and process text from a webpage, focusing on <p> tags.

#     Parameters:
#     url (str): The URL of the webpage to scrape.

#     Returns:
#     str: The cleaned-up text extracted from the web page, or an empty string if an issue occurs during scraping.
#     """
    
#     # returns class attributes of an HTML element or "NOCLASS" if it doesn't have any.
#     def get_class(p):
#         return ''.join(p['class']) if p.has_attr('class') else "&&NOCLASS&&"

#     p_dict = defaultdict(list)
#     try:
#         for element in soup.find_all():
#             if element.has_attr('class') and any("caption" in class_name.lower() or "newsletter" in class_name.lower() 
#                                                  for class_name in element['class']):
#                 element.extract()
#         # finds all <p> tags
#         paragraphs = soup.find_all('p')

#         for p in paragraphs:
#             if len(p.text.split(' ')) > 5:
#                 p_dict[get_class(p)].append(p.text)

#         # Find the class with the longest list of paragraphs
#         longest_key = max(p_dict, key=lambda k: len(p_dict[k]))

#         # Combine and clean text
#         full_text = re.sub(r'\s+', ' ', ' '.join(p_dict[longest_key])).strip()

#     except Exception as e:
#         # print("Exception "  + str(e) + " occurred for url: " + url)
#         return ""


#     return full_text
